# TensorBoard

#### 請搭配教學影片服用
#### Tutorial video: https://www.youtube.com/watch?v=SDeQRRRMUHU

#### 延伸上一次練習程式碼

In [1]:
#import tensorflow, numpy模組
import tensorflow as tf
import numpy as np

#import matplotlib 資料視覺化模組
#import matplotlib.pyplot as plt

In [2]:
#重要!!!!!
#清理顯卡內存
#防止顯卡內存不足報錯&提高速度
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  

In [3]:
#定義add_layer功能(添加神經層)
#inputs, in_size, out_size, activation_funchion=None激勵函數
def add_layer(inputs, in_size, out_size, activation_function=None):
    with tf.name_scope('layer'): #給予整個"神經層"名字為layer
        with tf.name_scope('weights'): #給予權重名字為weights
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W') #給予權重參數名字為W
        with tf.name_scope('biases'): #給予誤差名字為biases
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b') #給予誤差參數名字為b
        with tf.name_scope('Wx_plus_b'): #給予方程名字為Wx_plus_b
            Wx_plus_b = tf.matmul(inputs, Weights) + biases #y=wx+b
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

In [4]:
#輸入資料
x_data = np.linspace(-1, 1, 300)[:, np.newaxis] #生成300個區間為[-1,1]的數值, 然後縱向生成一個維度 #維度為一
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

#預留輸入給sess.run()進行訓練
with tf.name_scope('inputs'): #給予整個輸入名字為inputs
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input') #給予xs名字為x_input
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input') #給予ys名字為y_input

In [5]:
#新增一層神經網路l1(隱藏層)
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
#inputs, in_size, out_size, activation_function
#輸入數據, 輸入數據input data size, 隱藏層神經元數量, 激勵函數

#新增一層輸出層prediction
prediction = add_layer(l1, 10, 1, activation_function=None)
#inputs, in_size, out_size, activation_function
#輸入數據  *來自l1隱藏層=xs, 輸入數據size  *隱藏層size, 神經元輸出數量, 激勵函數

#評估 prediction值 與 y_data真實值的差別  (y_data輸入=ys)-prediction #方差
with tf.name_scope('loss'):#給予損失名字為loss
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),reduction_indices=[1])) #方差
    
with tf.name_scope('train'):#給予訓練名字為train
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) #優化器 #優化函數:梯度下降 #0.1為Learning rate 

init = tf.global_variables_initializer() #初始化/激活所有變量(Variable) #must have if define variable
sess = tf.Session()

writer = tf.summary.FileWriter("logs/", sess.graph) #設定Tensorboard畫圖儲存路徑及定義sess.graph畫圖功能

sess.run(init)

### Training

In [6]:
#!tensorboard --logdir=D:\Exercise\logs